# Entitlements in TOC-eligible parcels

In [1]:
import boto3
import geopandas as gpd
import intake
import numpy as np
import os
import pandas as pd

import utils
import laplan

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

In [3]:
FULL_PREFIX = list(laplan.pcts.VALID_PCTS_PREFIX)
remove_prefix = ["ENV", "PAR", "ADM"]
prefix = [x for x in FULL_PREFIX if x not in remove_prefix]

pcts = catalog.pcts.read()
pcts = laplan.pcts.subset_pcts(pcts,
                              start_date="2017-10-01",
                              prefix_list=prefix,
                              get_dummies=True)
pcts = laplan.pcts.drop_child_cases(pcts, keep_child_entitlements=True)

## Process PCTS
* Won't know which AINs are used in PCTS. Keep all the CASE_NBR-AINs but have a way to identify how many obs to drop later on
* Join parcels to zoning eligible zones
* Want all the entitlements (TOC or non-TOC) after 10/2017 in the TOC-eligible parcels, and then we can see what activity has occurred

In [3]:
def subset_pcts():   
    # Import PCTS - use function to subset
    FULL_PREFIX = list(laplan.pcts.VALID_PCTS_PREFIX)
    remove_prefix = ["ENV", "PAR", "ADM"]
    prefix = [x for x in FULL_PREFIX if x not in remove_prefix]
    
    pcts = catalog.pcts.read()
    pcts = laplan.pcts.subset_pcts(pcts,
                                  start_date="2017-10-01",
                                  prefix_list=prefix,
                                  get_dummies=True)
    pcts = laplan.pcts.drop_child_cases(pcts, keep_child_entitlements=True)
    
    column_index = pcts.columns.get_loc("GEOID")
    pcts = pcts.iloc[:, 0:column_index]
    
    # Import parcels
    parcels = catalog.crosswalk_parcels_tracts.read()
    
    # Import parcels with zoning crosswalk
    parcels_with_zoning = utils.download_geoparquet("parcels_joined_zones.parquet", 
                                                    S3_path = "gis/intermediate/")

    # Import parcels with zoning
    eligible_zones = ['R2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5', 
                  'RD1.5', 'RD2', 'RD3', 'RD4', 'RD5', 'RD6', 
                  'C1', 'C2', 'C4', 'C5']

    eligible_zoning = (parcels_with_zoning[parcels_with_zoning.zone_class.isin(eligible_zones)]
                        [["uuid", "zone_class"]]
                      )
    
    # There are duplicates in eligible_zoning, which means same uuid associated with 2 zone_classes
    # Pick more restrictive zone to keep
    eligible_zoning = eligible_zoning.assign(
        zone_order = eligible_zoning.zone_class.map(utils.ZONE_CLASS_ORDER)
    )

    eligible_zoning = (eligible_zoning.sort_values(["uuid", "zone_order"])
                       .drop_duplicates(subset = "uuid", keep = "first")
                       .drop(columns = "zone_order")
                      )
    
    # Merge parcel crosswalk with parcels in eligible zones
    # Inner join: drop those that aren't in eligible zones
    parcels_eligible_zones = pd.merge(parcels, eligible_zoning, 
                    on = "uuid", how = "inner", validate = "m:1")
    
    
    # Merge PCTS with parcel info to see which TOC Tier and zone class is associated with each 
    m1 = pd.merge(pcts, parcels_eligible_zones, on = 'AIN', how = 'inner', validate = 'm:1')   
    
    # THere are more columns than we need, and differences in extraneous columns gives us duplicates
    # Ex: APPEAL_DECISION_DATE, etc
    keep = ['CASE_ID', 'CASE_NUMBER', 'FILE_DATE', 'APPLICATION_ID',
            'CASE_SEQUENCE_NUMBER', 'CASE_YEAR_NUMBER', 'PARENT_CASE_ID',
            'CASE_ACTION_ID', 'AIN', 'PROJECT_DESCRIPTION',
            'uuid', 'x', 'y', 'num_AIN', 'TOC_Tier',
            'GEOID', 'total_AIN', 'pct_toc_AIN', 'toc_AIN', 'zone_class', 
           ]
    
    m2 = (m1[keep]
          .sort_values(["CASE_ID", "uuid", "AIN"])
          .drop_duplicates(subset=["CASE_ID", "CASE_NUMBER", "uuid", "zone_class"])
          .reset_index(drop=True)
         )
    
    # Check for duplicates on uuid-CASE_ID...
    # If it touches more than 20 parcels...it'll get dropped as outlier?
    # We are also only looking at unique CASE_IDs in analysis.
    m3 = m2.copy()
    m3["obs"] = m3.groupby(["CASE_ID", "uuid"]).cumcount() + 1
    m3["max_obs"] = m3.groupby(["CASE_ID", "uuid"])["obs"].transform("max")
    print("Duplicates on CASE_ID-uuid")
    print(m3.max_obs.value_counts())
    
    return m2

In [4]:
def more_pcts_processing(df): 
    # We care about TOC vs non-TOC entitlements
    df = df.assign(
        is_TOC = df.CASE_NUMBER.str.contains("TOC").astype(int),
    )
    
    # Subset by CASE_ACTION_ID -- let's use all cases for now (but approved cases are 1, 2, 11)
    # We have some NaN CASE_ACTION_IDs, so we won't subset at all
    
    # At this point, no more duplicates by PARENT_CASE - AIN combination
    
    # Do check and drop cases that apply to 20+ parcels (similar to D1 notebook)
    big_cases = (df.groupby('CASE_ID')
                 .agg({'uuid': 'count'})
                 .reset_index()
                )
    
    big_cases = big_cases[big_cases['uuid'] >= 20]
    df = df[~df.CASE_ID.isin(big_cases.CASE_ID)]
        
    return df

In [5]:
df1 = subset_pcts()   
df2 = more_pcts_processing(df1)

Duplicates on CASE_ID-uuid
1    48430
Name: max_obs, dtype: int64


## Create parcel-level df to use for TOC analysis
Use a df where each observation is case-AIN for TOC analysis. The TOC analysis naturally will count a case multiple times, depending on whether that case overlaps with a bus stop boundary, rail station boundary, etc. Bus stops and rail stations are naturally located near each other for transfer points, so counting a TOC case toward a particular bus line, rail line, or rail station naturally counts each TOC case multiple times over.

In [6]:
# This sub-function is used to reshape our df to parcel-level or tract-tier-level.
def groupby_make_wide(df, group_cols):
    # Group by a list of columns and count unique cases 
    # Group by case-tract-tier, case-AIN, etc
    group_cols2 = group_cols + ['is_TOC']
    
    df2 = (df.groupby(group_cols2)
           .agg({'CASE_ID':'count'})
           .rename(columns = {'CASE_ID': 'num_cases'})
           .reset_index()) 

    # Make wide
    df2 = df2.assign(
        num_TOC = df2.apply(lambda row: row.num_cases if row.is_TOC == 1 else np.nan, axis = 1),
        num_nonTOC = df2.apply(lambda row: row.num_cases if row.is_TOC == 0 else np.nan, axis = 1)
    )
    
    # If there are multiple obs for the same AIN, fill the NaNs with the max from the other column 
    # Then, drop duplicates
    df2 = df2.assign(
        num_TOC = df2.num_TOC.fillna(df2.groupby(group_cols)['num_TOC'].transform('max')),
        num_nonTOC = df2.num_nonTOC.fillna(df2.groupby(group_cols)['num_nonTOC'].transform('max'))
    )
    
    # Drop duplicates and clean up
    group_cols3 = group_cols + ['num_TOC', 'num_nonTOC']
    df3 = df2.drop_duplicates(subset = group_cols3)
    
    df3 = (df3.assign(
            num_TOC = df3.num_TOC.fillna(0).astype(int),
            num_nonTOC = df3.num_nonTOC.fillna(0).astype(int)
        ).drop(columns = ['is_TOC', 'num_cases'])
    )
    
    return df3

In [7]:
def make_parcel_level_df(df):
    """
    Ignore the fact that the same case can touch multiple parcels...
    TOC Tiers analysis will naturally count multiple times, because tiers coming from
    bus stops or rail lines will all overlap each other. 
    That analysis uses parcel-level PCTS combined with transit-stop-level TOC Tiers info.
    
    Reshape and make a parcel-level df to be used in later notebooks.
    But, we should drop duplicates based on uuid, so that it is truly the unique physical parcel.
    Create the toc_parcels_with_entitlements.geojson file.
    """
    
    keep_col = ['CASE_NUMBER', 'CASE_ID', 'CASE_ACTION_ID', 'FILE_DATE', 
                'uuid', 'AIN', 'x', 'y', 'TOC_Tier', 'zone_class', 'is_TOC']
    
    # Drop duplicates based on uuid
    df = (df[keep_col]
          .sort_values(["uuid", "AIN"])
          .drop_duplicates(subset = "uuid", keep = "first")
          .reset_index(drop=True)
         )
         
    # Make into parcel-level df and make wide
    parcel_level = ['uuid', 'AIN', 'x', 'y', 'TOC_Tier', 'zone_class']
    df2 = groupby_make_wide(df, parcel_level)
        
    # Make gdf
    df3 = gpd.GeoDataFrame(
        df2,
        geometry = gpd.points_from_xy(df2.x, df2.y), 
        crs = "EPSG:4326"
    ).drop(columns = ["x", "y"])
    
    return df3

In [8]:
toc_parcels_with_entitlements = make_parcel_level_df(df2)

file_name = "toc_eligible_parcels_with_entitlements.geojson"

toc_parcels_with_entitlements.to_file(driver = 'GeoJSON', filename = f'../gis/intermediate/{file_name}')

s3.upload_file(f'../gis/intermediate/{file_name}', bucket_name, f'gis/intermediate/{file_name}')

## Check if cases span multiple tracts / tiers / zone_classes 
For summary stats, we don't want a parcel-level df. We want to try and get as close as possible to the unique case. If we count a case multiple times simply because it touches multiple parcels, we'll overcount by a lot. 

**Outputs below show that a case can span multiple zone_classes within a tract-tier. Infrequent, but can happen.
A case will not span multiple tiers within a tract.**

### Pare down case to be unique to tract-tier-zone_class for descriptive stats
For descriptive summary stats:
* A row should be unique to case-tract-tier-zone_class
* Should we store how many parcels a case (case-tract-tier-zone) applies to by TOC case or not? Let's skip for now, unless there's a real need, we can add it later.
* When this df gets aggregated to tier or zone, a case that does span multiple tiers or multiple zones will be counted multiple times. This is ok. 

In [9]:
def count_obs(df, group_cols):
    df['obs'] = df.groupby(group_cols).cumcount() + 1
    df['max_obs'] = df.groupby(group_cols)["obs"].transform("max")
    print(df.max_obs.value_counts())

In [10]:
group_cols = ["CASE_ID", "GEOID", "TOC_Tier", "zone_class"]
print("# parcels for each CASE_ID")
count_obs(df2, group_cols)
# So a CASE_ID can be applied up to 8 different AINs within the same tract-tier-zone.

display(df2[df2.max_obs==8][["CASE_NUMBER", "AIN", "TOC_Tier", "zone_class", "GEOID"]].head(8))

# parcels for each CASE_ID
1     2210
2      656
3      270
4      236
5      155
6       96
11      33
7       28
8       24
9        9
Name: max_obs, dtype: int64


,CASE_NUMBER,AIN,TOC_Tier,zone_class,GEOID
103,ZA-2017-4225-CUW,6069007014,3,R2,06037242700
104,ZA-2017-4225-CUW,6069012013,3,R2,06037242700
105,ZA-2017-4225-CUW,6069012012,3,R2,06037242700
106,ZA-2017-4225-CUW,6069007016,3,R2,06037242700
107,ZA-2017-4225-CUW,6069007010,3,R2,06037242700
108,ZA-2017-4225-CUW,6069007012,3,R2,06037242700
109,ZA-2017-4225-CUW,6069007008,3,R2,06037242700
110,ZA-2017-4225-CUW,6069012023,3,R2,06037242700


In [11]:
keep = ["CASE_ID", "CASE_NUMBER", "TOC_Tier", "GEOID", "zone_class", "is_TOC"]
no_dups_by_tract_tier_zone = (df2[keep].drop_duplicates()
                              .sort_values(["GEOID", "TOC_Tier", "zone_class"])
                              .reset_index(drop=True)
                             )

group_cols = ["CASE_ID", "GEOID", "TOC_Tier"]
print("# zone_classes for each CASE_ID")
count_obs(no_dups_by_tract_tier_zone, group_cols)
# A couple of cases touch 2 different zone_classes within same tract

display(no_dups_by_tract_tier_zone[no_dups_by_tract_tier_zone.max_obs==3])

# zone_classes for each CASE_ID
1    2660
2      82
3       3
Name: max_obs, dtype: int64


,CASE_ID,CASE_NUMBER,TOC_Tier,GEOID,zone_class,is_TOC,obs,max_obs
1587,218745,CPC-2018-617-DB-SPR,3,06037213100,C2,0,1,3
1589,218745,CPC-2018-617-DB-SPR,3,06037213100,R3,0,2,3
1590,218745,CPC-2018-617-DB-SPR,3,06037213100,R4,0,3,3


In [12]:
keep = ["CASE_ID", "CASE_NUMBER", "TOC_Tier", "GEOID", "is_TOC"]
no_dups_by_tract_tier = (df2[keep].drop_duplicates()
                              .sort_values(["GEOID", "TOC_Tier"])
                              .reset_index(drop=True)
                             )

group_cols = ["CASE_ID", "GEOID", "TOC_Tier"]
print("# tiers for each CASE_ID")
count_obs(no_dups_by_tract_tier, group_cols)
# Cases always fall within the same tract-tier.

# tiers for each CASE_ID
1    2702
Name: max_obs, dtype: int64


In [13]:
keep = ["CASE_ID", "CASE_NUMBER", "GEOID", "is_TOC"]
no_dups_by_tract = (df2[keep].drop_duplicates()
                              .sort_values(["GEOID"])
                              .reset_index(drop=True)
                             )

group_cols = ["CASE_ID", "GEOID"]
print("# tracts for each CASE_ID")
count_obs(no_dups_by_tract, group_cols)
# Cases always fall within the same tract.

# tracts for each CASE_ID
1    2576
Name: max_obs, dtype: int64


## Create tract-tier-zone_class-level df
Now that the data exploration made clear just how CASE_ID can span these various geographies, we can get our df ready for summary stats.

In [14]:
def unique_to_tract_tier_zone(df):    
    # Only keep 1 case for each tract-tier-zone
    keep_cols = ["CASE_ID", "TOC_Tier", "GEOID", "CASE_NUMBER", 
                 "CASE_SEQUENCE_NUMBER", "CASE_YEAR_NUMBER", 
                 "CASE_ACTION_ID", "FILE_DATE", 
                 "PARENT_CASE_ID", "PROJECT_DESCRIPTION", "zone_class", "is_TOC"]
    
    df2 = (df[keep_cols].drop_duplicates()
                   .reset_index(drop=True)
                  )
        
    # Get counts of num_TOC and num_nonTOC by tract-tiers-zone
    tract_tier_level = ['GEOID', 'TOC_Tier', 'zone_class']
    df3 = groupby_make_wide(df2, tract_tier_level)
    
    return df3

In [15]:
df = unique_to_tract_tier_zone(df2)
df.head()

,GEOID,TOC_Tier,zone_class,num_TOC,num_nonTOC
0,06037101210,0,C2,0,5
1,06037101210,0,R3,0,2
2,06037101210,0,RD1.5,0,1
3,06037101220,0,C2,0,1
4,06037101300,0,C2,0,2


## Summary stats by parcels

In [16]:
toc_parcels = toc_parcels_with_entitlements[toc_parcels_with_entitlements.num_TOC > 0]
non_toc_parcels = toc_parcels_with_entitlements[toc_parcels_with_entitlements.num_nonTOC > 0]
have_both_parcels = toc_parcels_with_entitlements[(toc_parcels_with_entitlements.num_TOC > 0) & 
                                                  (toc_parcels_with_entitlements.num_nonTOC > 0)]

print(f'# parcels: {len(toc_parcels_with_entitlements)}')
print(f'# parcels with TOC entitlements: {len(toc_parcels)}')
print(f'# parcels with non TOC entitlements: {len(non_toc_parcels)}')
print(f'# parcels with both TOC and non TOC entitlements: {len(have_both_parcels)}')
print(f'double check sum: {len(toc_parcels) + len(non_toc_parcels) - len(have_both_parcels)}')

# parcels: 3064
# parcels with TOC entitlements: 507
# parcels with non TOC entitlements: 2557
# parcels with both TOC and non TOC entitlements: 0
double check sum: 3064


In [17]:
print(f'% parcels with TOC entitlements: {len(toc_parcels) / len(toc_parcels_with_entitlements)}')
print(f'% parcels with non TOC entitlements: {len(non_toc_parcels) / len(toc_parcels_with_entitlements)}')
print(f'% parcels with both entitlements: {len(have_both_parcels) / len(toc_parcels_with_entitlements)}')

% parcels with TOC entitlements: 0.16546997389033943
% parcels with non TOC entitlements: 0.8345300261096605
% parcels with both entitlements: 0.0


In [18]:
toc_parcels.zone_class.value_counts()

C2       205
R3       171
R4        90
C4        23
RD1.5      9
RD2        3
R5         3
RAS4       2
R2         1
Name: zone_class, dtype: int64

In [19]:
non_toc_parcels.zone_class.value_counts()

C2       1086
C4        361
R3        279
RD1.5     257
R2        169
RD2       135
R4         86
C1         55
R5         39
C5         32
RD3        26
RAS4       19
RAS3        5
RD5         4
RD4         2
RD6         2
Name: zone_class, dtype: int64

## Breakdown by TOC Tiers
Observations are at the case-tract-tier level.

Tracts are not mutually exclusive! A tract can span multiple tiers (ex: tract is partly in tier 3, partly in tier 4). If there are TOC or Non-TOC entitlements happening, that same tract would show up as an eligible tract in tier 3, and again in tier 4.

In [20]:
def summarize_by_tiers(df):
    df2 = (df.groupby(['TOC_Tier'])
           .agg({'GEOID': 'nunique', 
                 'num_TOC':'sum', 
                 'num_nonTOC':'sum'})
           .rename(columns = {'GEOID': 'num_TOC_eligible_tracts'})
           .reset_index()
          )
    
    for i in ['TOC', 'nonTOC']:
        new_col = f'pct_{i}'
        numerator = f'num_{i}'
        df2[new_col] = df2[numerator] / (df2.num_TOC + df2.num_nonTOC)
        
    return df2

by_tiers = summarize_by_tiers(df)
by_tiers

,TOC_Tier,num_TOC_eligible_tracts,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC
0,0,356,47,903,0.049474,0.950526
1,1,173,52,410,0.112554,0.887446
2,2,149,58,260,0.182390,0.817610
3,3,251,181,670,0.212691,0.787309
4,4,45,20,144,0.121951,0.878049


## Breakdown by Zone Class
Observations are at the case-tract-tier-zone_class level. Recall: there are some cases, same CASE_ID, but span different zone_class within the tract. That case would show up multiple times in our aggregation.

Tracts are not mutually exclusive! A tract can obviously have multiple zone classes (ex: tract is partly R3, partly R4). If there are TOC or Non-TOC entitlements happening, that same tract would show up as an eligible tract in R3 and R4.

In [21]:
def summarize_by_zones(df):
    df2 = (df.groupby('zone_class')
           .agg({'GEOID':'nunique', 
                 'num_TOC':'sum', 
                 'num_nonTOC':'sum'})
           .rename(columns = {'GEOID': 'num_TOC_eligible_tracts'})
           .reset_index()
          )
    
    for i in ['TOC', 'nonTOC']:
        new_col = f'pct_{i}'
        numerator = f'num_{i}'
        df2[new_col] = df2[numerator] / (df2.num_TOC + df2.num_nonTOC)
        
    return df2

by_zones = summarize_by_zones(df)
by_zones

,zone_class,num_TOC_eligible_tracts,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC
0,C1,27,0,44,0.000000,1.000000
1,C2,388,135,933,0.126404,0.873596
2,C4,113,18,357,0.048000,0.952000
3,C5,2,0,37,0.000000,1.000000
4,R2,49,1,177,0.005618,0.994382
5,R3,175,129,258,0.333333,0.666667
6,R4,73,59,77,0.433824,0.566176
7,R5,10,3,39,0.071429,0.928571
8,RAS3,3,1,3,0.250000,0.750000
9,RAS4,14,1,17,0.055556,0.944444


In [22]:
writer = pd.ExcelWriter('../outputs/toc_charts.xlsx', engine = 'xlsxwriter')

by_tiers.to_excel(writer, sheet_name = 'entitlements_by_tier')
by_zones.to_excel(writer, sheet_name = 'entitlements_by_zone')

writer.save()